# Lesson 2 - BOLDgeometry

The `BOLDgeometry` module contains the objects which define voxels in simulations. To begin we first import the module as follows (and NumPy for array creation):

In [ ]:
from boldswimsuite import BOLDgeometry, BOLDvessel
import numpy as np

## Creating Continuous Voxels

A continuous voxel consists of a list of vessel objects, with a voxel size and B0 magnetic field strength.

>Note: The voxel is always centered around zero.

### Using the default constructor

Using what we learned in Lesson 1, we can create some `InfiniteCylinder3D` vessels that we will use to construct a voxel.

In [ ]:
vessel1 = BOLDvessel.InfiniteCylinder3D(
    diameter=1, #mm
    theta=-np.pi/3, #radians
    phi=np.pi/4,
    origin=np.array([0, -1., 0.]), #mm
    dchi=3e-8, #cgs units
    permeation_probability=0, #probability
    label='vein'
)

vessel2 = BOLDvessel.InfiniteCylinder3D(
    diameter=1, #mm
    theta=5*np.pi/9, #radians
    phi=-np.pi/5,
    origin=np.array([1., 1., 0.]), #mm
    dchi=3e-8, #cgs units
    permeation_probability=0, #probability
    label='artery'
)

vessels = [vessel1, vessel2]

The default constructor of `ContinuousVoxel3D` can be used to transform our list of vessels into a voxel. The relevant arguments are:
- `size` : float, the side length of the voxel (in mm). 
- `B0` : float, the B0 magnetic field strength (in Tesla).
- `vessels` : List[Vessel3D], a list of 3D vessel objects.

In [ ]:
continuous_voxel = BOLDgeometry.ContinuousVoxel3D(
    size=5, #mm
    B0=3, #T
    vessels=vessels
)

### Using the random generator

We can also use the class method `from_random` to create a randomly generated voxel from a series of parameters.

In [ ]:
random_continuous_voxel = BOLDgeometry.ContinuousVoxel3D.from_random(
    size=5,
    CBV=0.02,
    B0=3,
    labels=['vein', 'artery'],
    weights={
        'vein':1, 
        'artery':1
    },
    diameter_distributions={
        'vein': [0.2, 0.3, 0.4], 
        'artery': [0.3, 0.4, 0.5]
    },
    dchis={
        'vein': 3e-8,
        'artery': 4e-8
    },
    permeation_probabilities={
        'vein': 0, 
        'artery': 0.1
    },
    vessel_type='cylinder',
    allow_vessel_intersection=True,
    seed=0,
    progressbar=True
)

print(random_continuous_voxel)

> Note: the CBV of voxels created using `from_random` will not be exactly what has been provided to the method. Having more vessels (either by increasing the voxels's `size` or by increasing the `CBV`), will reduce the possibility of having a large error in the CBV.

## Basic Methods for Voxels

The primary methods for voxels mirror that of vessels, namely `vessel_indices_from_positions` and `dBz_vessel_indices_from_positions`.

The first takes a positions array and returns an array indicating the vessel index for each position. A vessel index of 0 means extravascular, while any integer above 0 means intravascular. The value of the integer represents which vessel the position occupies.

In [ ]:
positions = np.array(
    [[0.1, -1.1, 0.1], #(mm)
     [ 2.,   2.,  2.],
     [1.1,  1.1, 0.1]]
)

vessel_indices = continuous_voxel.vessel_indices_from_positions(positions)

print(f'vessel indices: {vessel_indices}')

The second method takes an array of positions and returns both the vessel indices and an array of dBz.

In [ ]:
dBz, vessel_indices = continuous_voxel.dBz_vessel_indices_from_positions(positions)

print(f'dBz: {dBz}')
print(f'vessel indices: {vessel_indices}')

In practice, these are rarely be used by the user. Rather, a full simulation will make use of these methods automatically during run time (as shown in later lessons).

A more practical method is `get_CBV`, which calculates an fairly accurate approximation of the CBV. This approximation is very accurate for large, populated voxels.

In [ ]:
CBV = continuous_voxel.get_CBV()

print(f'CBV: {CBV}')

### Visualizing Continuous Voxels

All voxels can be visualized by using the `show` method. For 3D voxels, this will open a window running Mayavi, a 3D plotting package. For 2D voxels, this will open a window running Matplotlib, another plotting package. The `show` method for 3D voxels takes camera position settings so that repeatable voxel views can be generated:
- `azimuth`: float, the azimuth angle of the camera from the voxel.
- `elevation`: float, the elevation angle of the camera from the voxel.
- `distance`: float, distance from the center of the voxel to the camera (mm).

These all have default values, so we can choose to omit these parameters.

These parameters do not exist for the 2D continuous voxel `show` method as it will always show a 2D image of the whole voxel.

In [ ]:
continuous_voxel.show()

## Creating Discrete Voxels

Discrete voxels are objects defined by 5 variables:
- `N` : int, the number of discrete points along the voxel edges. Therefore a 3D discrete voxel is represented on a (N,N,N) grid and a 2D discrete voxel is represented on a (N,N) grid.
- `size` : float, the side length of the voxel (in mm). 
- `vessel_index_grid` : np.ndarray, an integer array of shape (N,N,N) (or (N,N) in 2D). It serves as a discretized representation of the voxel space, indicating where the intravascular and extravascular spaces are located. A value of 0 represents the extravascular space and positive integers (1,2,3...) represent intravascular space. Different positive integers represent different vessels, or vessel types, which can be associated with different properties. The integer associated to a specific vessel is called its 'vessel index'.
- `permeation_probability_list` : List[float], a list of probabilities (between 0 and 1) which indicate the probability for Monte Carlo spins to permeate in and out of the vessels. The first item in the list corresponds to the permeation probability of all vessels with a vessel index of 1, the second item in the list corresponds to the permeation probability of all vessels with a vessel index of 2, and so on for any additional vessel index. The extravascular space does not have a permeation probability, so a vessel index of 0 does not have an associated permeation probability in the list.
- `dBz_grid` : np.ndarray, a float array of shape (N,N,N) (or (N,N) in 2D). It represents the magnetic field offset space (in Tesla).

As a result, the default constructor requires these parameters (excluding `N` as it can be inferred from both `vessel_index_grid` and `dBz_grid`). However it can be difficult to manually generate `dBz_grid` or even `vessel_index_grid`, so multiple alternate constructors are provided to generate the discrete voxel.

### Converting Continuous Voxels to Discrete Voxels

One of the ways to create discrete voxels is by using the `from_continuous_anaytical` class method, which takes for argument `N` and a continuous voxel (`ContinuousVoxel3D` for `DiscreteVoxel3D`, and `ContinuousVoxel2D` for `DiscreteVoxel2D`). We will use the continuous voxel we created earlier to show this:

In [ ]:
discrete_voxel = BOLDgeometry.DiscreteVoxel3D.from_continuous_analytical(
    N=100, 
    voxel=continuous_voxel
)

### Visualizing Discrete Voxels

Much like continuous voxels, discrete voxels can be visualized using the `show` method. Although there is an additional parameter:
- `show_dBz`: bool, if True, will show the dBz field offset rather than the vessel mask. By default, False.
- `azimuth`: float, the azimuth angle of the camera from the voxel.
- `elevation`: float, the elevation angle of the camera from the voxel.
- `distance`: float, distance from the center of the voxel to the camera (mm).

This additional parameter is found in both the 2D and 3D discrete voxels.

In [ ]:
discrete_voxel.show(show_dBz=False)

In [ ]:
discrete_voxel.show(show_dBz=True)

We can use also use the same methods as shown for continuous voxels:

In [ ]:
CBV = discrete_voxel.get_CBV()

print(f'CBV: {CBV}')

positions = np.array(
    [[0.1, -1.1, 0.1], #(mm)
     [ 2.,   2.,  2.],
     [1.1,  1.1, 0.1]]
)

dBz, vessel_indices = discrete_voxel.dBz_vessel_indices_from_positions(positions)

print(f'dBz: {dBz}')
print(f'vessel indices: {vessel_indices}')

The previous method of converting a continuous voxel into a discrete voxel used the analytical equations to determine the magnetic field offset, since the continuous voxel can provide them. However, we can also use FFT methods to calculate the magnetic field offset `dBz_grid` from a pre-generated `vessel_index_grid`. This allows the discrete voxels to accomodate arbitrary perturber geometry. Using the `from_continuous_FFT` class method of the discrete voxel we can once again convert continuous voxels to discrete voxels, but this time using the FFT method to calculate the magnetic field offset.

>Note: the FFT method is only available for 3D discrete voxels, currently.

This method has additional parameters, which are used during the FFT step and can help improve the accuracy of result. The additional parameters are:
- `padding` : int, amount of zero padding to add to each side of the voxel before the FFT step. The default is 0, which will cause wrapping of the field offset. Using a value of N/2 will completely remove the wrapping effect but is more computationally demanding.
- `extend` : bool, if True, will extend the vessels to the zero padding, but is more computationally demanding. Doing so creates a more accurate representation of the continuous voxel (e.g. infinite cylinders cannot be infinite in the discrete space, but extending the vessels will make them "more" infinite). Default is False.

In [ ]:
discrete_voxel_FFT = BOLDgeometry.DiscreteVoxel3D.from_continuous_FFT(
    N=100, 
    voxel=continuous_voxel,
    padding=50, #to remove wrapping effects
    extend=True
)

Once again we visualize the voxel with `show`.

In [ ]:
discrete_voxel_FFT.show(show_dBz=False)

In [ ]:
discrete_voxel_FFT.show(show_dBz=True)

A disadvantage of FFT is that it creates a noise around the vessel edges, unlike the analytical method.

### Creating a Discrete Voxel from User-Defined Perturber Geometry

The FFT method can also work with user-defined perturber geometries, as long as a `vessel_index_grid` is provided, along with some additional parameters. For this example we will manually make two spheres positioned in the voxel.

In [ ]:
N = 100
size = 5.0

sphere_radius = 0.8
spatial_range = np.linspace(-size/2, size/2, N)
X, Y, Z = np.meshgrid(spatial_range, spatial_range, spatial_range)

sphere1_mask = np.sqrt((X+1)**2 + (Y+1)**2 + Z**2) < sphere_radius
sphere2_mask = np.sqrt((X-1)**2 + (Y-1)**2 + Z**2) < sphere_radius

vessel_index_grid = np.zeros((N, N, N), dtype=int)
vessel_index_grid[sphere1_mask] = 1 # we assign the first sphere a vessel index of 1
vessel_index_grid[sphere2_mask] = 2 # we assign the second sphere a vessel index of 2

>Note: Different vessels can have the same vessel index. This index is used to assign parameters to the vessels (such as magnetic susceptibility and permeation probability), so vessels with the same parameters can share the same index.

Now that we made our `vessel_index_grid`, we can create a 3D discrete voxel with it, using the `from_vessel_index_grid_FFT` class method. This method has additional arguments to assign parameters to the vessels:
- `dchis` : Union[List[float], np.ndarray], The susceptibility difference between the intravascular and extravascular space. Can be a list of magnetic susceptibility differences where the first item in the list corresponds to the magnetic susceptibility difference of of all vessels with a vessel index of 1, the second item in the list corresponds to the magnetic susceptibility difference of all vessels with a vessel index of 2, and so on for any additional vessel index. Can also be a float array of shape (N, N, N), indicating the magnetic susceptibility difference at each point in space. Units are in cgs.
- `permeation_probability_list` : List[float], a list of probabilities (between 0 and 1) which indicate the probability for Monte Carlo spins to permeate in and out of the vessels. The first item in the list corresponds to the permeation probability of all vessels with a vessel index of 1, the second item in the list corresponds to the permeation probability of all vessels with a vessel index of 2, and so on for any additional vessel index.
- `size` : float, the side length of the voxel (in mm). 
- `B0` : float, the B0 magnetic field strength (in Tesla).
- `padding` : int, amount of zero padding to add to each side of the voxel before the FFT step. The default is 0, which will cause wrapping of the field offset. Using a value of N/2 will completely remove the wrapping effect but is more computationally demanding.

We see that the `dchis` argument has 2 possible inputs, either a list or an array. Using the list can be more convenient with voxels containing a small number of different susceptibilities. Using the array is more convenient when we have direct access to the susceptibility grid.

First we will use the list option:

In [ ]:
spheres_voxel_FFT = BOLDgeometry.DiscreteVoxel3D.from_vessel_index_grid_FFT(
    vessel_index_grid=vessel_index_grid,
    dchis=[3e-8, 4e-8],
    permeation_probability_list=[0., 0.4],
    size=size,
    B0=3,
    padding=int(N/2)
)

In [ ]:
spheres_voxel_FFT.show(show_dBz=False)

In [ ]:
spheres_voxel_FFT.show(show_dBz=True)

Now we can try using the array option. We will first need to create the susceptibility grid, although for most use cases of this option the susceptibility grid will already be generated externally. We then give the function the array instead of a list, which produces the same result.

In [ ]:
sphere1_susceptibility = 3e-8
sphere2_susceptibility = 4e-8

# creating an array of zeros with the same shape as the vessel index grid
# we also specify the dtype to be float
dchi_grid = np.zeros_like(vessel_index_grid, dtype=float)

# assign the suscpetibilities of each sphere in the susceptibility grid
dchi_grid[vessel_index_grid == 1] = sphere1_susceptibility
dchi_grid[vessel_index_grid == 2] = sphere2_susceptibility

spheres_voxel_FFT = BOLDgeometry.DiscreteVoxel3D.from_vessel_index_grid_FFT(
    vessel_index_grid=vessel_index_grid,
    dchis=dchi_grid,
    permeation_probability_list=[0., 0.4],
    size=size,
    B0=3,
    padding=int(N/2)
)

In [ ]:
spheres_voxel_FFT.show(show_dBz=True)